In [1]:
import main_rnn, main_cnn
from icecream import ic
import numpy as np

In [2]:
from src.layer.dense import Dense
from src.layer.lstm import LSTM
from src.sequential import Sequential
from src.layer.detector import Detector
from src.layer.flatten import Flatten
from src.layer.pooling import Pooling
from src.layer.convolutional import Convolutional

## Model

In [3]:
window_size=3
model = Sequential()
model.add(LSTM(input_shape=(window_size, 5), hidden_cell_dim=64, return_sequences=False))
model.add(Dense(size=5, input_size=64, activation="linear"))

In [4]:
model.summary()

-------------------------------------
Layer	Output Shape	Param Count
lstm	(64, 1)		17920
dense	5		325
Total Params: 18245
-------------------------------------


## RNN Experiment

In [5]:
def perform_experiment(window_size=5):
    train_data_x, _ = main_rnn.get_data(window_size)

    model = Sequential()
    model.add(LSTM(input_shape=(window_size, 5), hidden_cell_dim=64, return_sequences=False))
    model.add(Dense(size=5, input_size=64, activation="linear"))
    predictions = model.run(inputs=train_data_x)

    ic(predictions)
    ic(predictions.shape)

    model.save_model(f"model-lstm-window-{window_size}.json")

### First Experiment

In [6]:
perform_experiment(4)

Finished processing data 500
Finished processing data 1000
Finished processing data 1500
Finished processing data 2000
Finished processing data 2500
Finished processing data 3000
Finished processing data 3500
Finished processing data 4000
Finished processing data 4500
Finished processing data 5000
Finished processing data 5500
Finished processing data 6000
Finished processing data 6500
Finished processing data 7000
Finished processing data 7500
Finished processing data 8000
Finished processing data 8500
Finished processing data 9000
Finished processing data 9500


ic| predictions: array([[36.00006497, 34.84617467, 37.38547231, 30.65414743, 29.86609375],
                        [35.99317811, 34.83897122, 37.37905304, 30.64810512, 29.86075089],
                        [35.99652988, 34.84248549, 37.38213848, 30.65107447, 29.86335223],
                        ...,
                        [36.00492336, 34.85146893, 37.39017661, 30.65835527, 29.86973515],
                        [36.00179465, 34.84810403, 37.38680709, 30.65570955, 29.86707223],
                        [36.00281872, 34.84919295, 37.38788532, 30.65657102, 29.86792673]])
ic| predictions.shape: (9641, 5)


### Second Experiment

In [7]:
perform_experiment(10)

Finished processing data 500
Finished processing data 1000
Finished processing data 1500
Finished processing data 2000
Finished processing data 2500
Finished processing data 3000
Finished processing data 3500
Finished processing data 4000
Finished processing data 4500
Finished processing data 5000
Finished processing data 5500
Finished processing data 6000
Finished processing data 6500
Finished processing data 7000
Finished processing data 7500
Finished processing data 8000
Finished processing data 8500
Finished processing data 9000
Finished processing data 9500


ic| predictions: array([[36.50191619, 27.74988205, 33.30564343, 31.80472772, 33.77352317],
                        [36.50191615, 27.74988201, 33.30564339, 31.80472768, 33.77352314],
                        [36.50191617, 27.74988203, 33.30564341, 31.8047277 , 33.77352315],
                        ...,
                        [36.50191621, 27.74988206, 33.30564345, 31.80472774, 33.77352319],
                        [36.50191621, 27.74988206, 33.30564345, 31.80472774, 33.77352319],
                        [36.50191619, 27.74988205, 33.30564344, 31.80472772, 33.77352318]])
ic| predictions.shape: (9635, 5)


### Third Experiment

In [8]:
perform_experiment(20)

Finished processing data 500
Finished processing data 1000
Finished processing data 1500
Finished processing data 2000
Finished processing data 2500
Finished processing data 3000
Finished processing data 3500
Finished processing data 4000
Finished processing data 4500
Finished processing data 5000
Finished processing data 5500
Finished processing data 6000
Finished processing data 6500
Finished processing data 7000
Finished processing data 7500
Finished processing data 8000
Finished processing data 8500
Finished processing data 9000
Finished processing data 9500


ic| predictions: array([[33.13306214, 36.9189834 , 29.81428983, 26.03596699, 33.28400286],
                        [33.13306214, 36.9189834 , 29.81428983, 26.03596699, 33.28400286],
                        [33.13306214, 36.9189834 , 29.81428983, 26.03596699, 33.28400286],
                        ...,
                        [33.13306214, 36.9189834 , 29.81428983, 26.03596699, 33.28400286],
                        [33.13306214, 36.9189834 , 29.81428983, 26.03596699, 33.28400286],
                        [33.13306214, 36.9189834 , 29.81428983, 26.03596699, 33.28400286]])
ic| predictions.shape: (9625, 5)


## CNN Experiment

In [10]:
train_data = main_cnn.get_data()
train_x, train_y = train_data["x"], train_data["y"]
train_x = np.asarray(train_x)

train_x = train_x.reshape((500, 3, 256, 256))

train_x = train_x.astype("float")
train_x /= 255

train_x = train_x[:3]

model = Sequential()
model.add(
    Convolutional(
        input_shape=(3, 256, 256),
        padding=0,
        filter_count=2,
        kernel_shape=(3, 3),
        stride=1,
    )
)
model.add(Detector(activation="relu"))
model.add(Pooling(size=(2, 2), stride=2,
            mode="max", input_shape=(254, 254, 2)))
model.add(Pooling(size=(5, 5), stride=5,
            mode="avg", input_shape=(127, 127, 2)))
model.add(Flatten(input_shape=(25, 25, 2)))
model.add(Dense(size=10, input_size=1250, activation="relu"))
model.add(Dense(size=1, input_size=10, activation="sigmoid"))
model.summary()

predictions = model.run(inputs=train_x)

ic(predictions)

-------------------------------------
Layer	Output Shape	Param Count
conv2d	(254, 254, 2)		4608
dtctr	same as input	0
max_pool	(126, 126, 2)	0
avg_pool	(24, 24, 2)	0
flatten	1250	0
dense	10		12510
dense	1		11
Total Params: 17129
-------------------------------------


ic| predictions: array([[0.93199036],
                        [0.93199036],
                        [0.93199036]])


array([[0.93199036],
       [0.93199036],
       [0.93199036]])